In [3]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

이미지 불러와서 numpy로 바꾸기

In [99]:
#keras로 이미지 불러오기 
from keras.preprocessing.image import img_to_array, load_img, array_to_img


#음식 데이터 to numpy  , Label = 0
data_fd = []
data_fd_label = []
for num in range(1,501):
    #classes = ['exterior', 'interior','food']
    image_file = './images/{}{}.jpg'.format('food', num)
    image_array = img_to_array(load_img(image_file))
    #print(image_array)
    
    data_fd.append(image_array)
    data_fd_label.append(0)

data_fd = np.array(data_fd)
data_fd_label = np.array(data_fd_label)

In [100]:
#실내 데이터 to numpy  , Label = 1

data_in = []
data_in_label = []
for num in range(1,501):
    #classes = ['exterior', 'interior','food']
    image_file = './images/{}{}.jpg'.format('interior', num)
    image_array = img_to_array(load_img(image_file))
    #print(image_array)
    
    data_in.append(image_array)
    data_in_label.append(1)

data_in = np.array(data_in)
data_in_label = np.array(data_in_label)
print(len(data_in))


500


In [97]:
print(np.array(data_in_label).shape)

(500,)


In [130]:
my_list = [[[1,2], [4,5]], [234,23]]
answer = sum(my_list, [])
answer

[[1, 2], [4, 5], 234, 23]

In [175]:
#실외 데이터 to numpy  , Label = 2
data_ex= []
data_ex_label= []
for num in range(1,2):
    #classes = ['exterior', 'interior','food']
    image_file = './images/{}{}.jpg'.format('exterior', num)
    image_array = img_to_array(load_img(image_file))
    #print(image_array)
    data_ex.append(image_array)
    
    data_ex_label.append(2)
    
data_ex = np.array(data_ex)
data_ex_label = np.array(data_ex_label)

훈련 비율은 0.85 : 0.15 로 설정합니다.

In [119]:
train_test_split = 0.85

train_data_fd = int(len(data_fd)*train_test_split)
train_data_in = int(len(data_in)*train_test_split)
train_data_ex = int(len(data_ex)*train_test_split)
#print(train_data_fd,train_data_in,train_data_ex)



#print(len(data_fd[:train_data_fd]))


#  X_train , X_test 만들기 
X_train_data_set = np.concatenate((data_fd[:train_data_fd], data_in[:train_data_in], data_ex[:train_data_ex]), axis= 0)
print(X_train_data_set.shape)
X_test_data_set =np.concatenate((data_fd[train_data_fd: ], data_in[train_data_in:], data_ex[train_data_ex:]))
print(X_test_data_set.shape)

y_train_data_set = np.concatenate((data_fd_label[:train_data_fd], data_in_label[:train_data_in], data_ex_label[:train_data_ex]))
y_test_data_set =np.concatenate((data_fd_label[train_data_fd: ], data_in_label[train_data_in:], data_ex_label[train_data_ex:]))
print(y_train_data_set.shape)
print(y_test_data_set.shape)

print(X_train_data_set)


(1275, 300, 300, 3)
(225, 300, 300, 3)
(1275,)
(225,)
[[[[184. 146. 109.]
   [185. 147. 110.]
   [185. 147. 110.]
   ...
   [ 73.  83.  75.]
   [ 71.  80.  75.]
   [ 72.  81.  76.]]

  [[174. 137.  95.]
   [175. 138.  96.]
   [177. 140.  98.]
   ...
   [ 74.  85.  77.]
   [ 67.  78.  72.]
   [ 66.  75.  70.]]

  [[158. 121.  77.]
   [159. 122.  78.]
   [160. 123.  78.]
   ...
   [ 64.  81.  71.]
   [ 60.  76.  66.]
   [ 64.  75.  67.]]

  ...

  [[ 10.   6.   3.]
   [ 14.   9.   6.]
   [ 16.   3.   0.]
   ...
   [184. 178. 166.]
   [176. 172. 161.]
   [173. 169. 158.]]

  [[ 11.   9.  10.]
   [ 14.   9.   6.]
   [ 23.  10.   2.]
   ...
   [179. 175. 163.]
   [178. 174. 163.]
   [175. 173. 161.]]

  [[ 23.  23.  25.]
   [ 16.  10.  10.]
   [ 24.  11.   3.]
   ...
   [179. 175. 163.]
   [179. 177. 165.]
   [179. 176. 167.]]]


 [[[ 29.  19.   7.]
   [ 30.  20.   8.]
   [ 28.  18.   6.]
   ...
   [126. 114. 102.]
   [125. 113. 101.]
   [128. 116. 104.]]

  [[ 29.  19.   7.]
   [ 31.  21. 

In [193]:
from sklearn.utils import shuffle
X_train_0, y_train_0 = shuffle(X_train_data_set, y_train_data_set, random_state=42)
X_test_0, y_test_0= shuffle(X_test_data_set, y_test_data_set, random_state=42)

In [194]:
def plot_loss_curve(history):
    plt.figure(figsize = (5,3))
    
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['train','test'], loc = 'upper right')
    plt.show
    

모델링

In [245]:
def image_load_data():
    #음식 데이터 to numpy  , Label = 0
    data_fd = []
    data_fd_label = []
    for num in range(1,501):
        #classes = ['exterior', 'interior','food']
        image_file = './images/{}{}.jpg'.format('food', num)
        image_array = img_to_array(load_img(image_file))
        #print(image_array)

        data_fd.append(image_array)
        data_fd_label.append(0)

    data_fd = np.array(data_fd)
    data_fd_label = np.array(data_fd_label)

    #실내 데이터 to numpy  , Label = 1

    data_in = []
    data_in_label = []
    for num in range(1,501):
        #classes = ['exterior', 'interior','food']
        image_file = './images/{}{}.jpg'.format('interior', num)
        image_array = img_to_array(load_img(image_file))
        #print(image_array)

        data_in.append(image_array)
        data_in_label.append(1)

    data_in = np.array(data_in)
    data_in_label = np.array(data_in_label)
    print(len(data_in))


    #실외 데이터 to numpy  , Label = 2
    data_ex= []
    data_ex_label= []
    for num in range(1,2):
        #classes = ['exterior', 'interior','food']
        image_file = './images/{}{}.jpg'.format('exterior', num)
        image_array = img_to_array(load_img(image_file))
        #print(image_array)
        data_ex.append(image_array)

        data_ex_label.append(2)

    data_ex = np.array(data_ex)
    data_ex_label = np.array(data_ex_label)

    train_test_split = 0.85

    train_data_fd = int(len(data_fd)*train_test_split)
    train_data_in = int(len(data_in)*train_test_split)
    train_data_ex = int(len(data_ex)*train_test_split)
    #print(train_data_fd,train_data_in,train_data_ex)



    #print(len(data_fd[:train_data_fd]))


    #  X_train , X_test 만들기 
    X_train_data_set = np.concatenate((data_fd[:train_data_fd], data_in[:train_data_in], data_ex[:train_data_ex]))
    print(X_train_data_set.shape)
    X_test_data_set =np.concatenate((data_fd[train_data_fd: ], data_in[train_data_in:], data_ex[train_data_ex:]))
    print(X_test_data_set.shape)

    y_train_data_set = np.concatenate((data_fd_label[:train_data_fd], data_in_label[:train_data_in], data_ex_label[:train_data_ex]))
    y_test_data_set =np.concatenate((data_fd_label[train_data_fd: ], data_in_label[train_data_in:], data_ex_label[train_data_ex:]))
    print(y_train_data_set.shape)
    print(y_test_data_set.shape)

    print(X_train_data_set)


    from sklearn.utils import shuffle
    X_train_0, y_train_0 = shuffle(X_train_data_set, y_train_data_set, random_state=42)
    X_test_0, y_test_0= shuffle(X_test_data_set, y_test_data_set, random_state=42)

    print(X_test.shape)
    
    return (X_train_0, y_train_0) ,( X_test_0, y_test_0)

image_load_data()

MemoryError: Unable to allocate 515. MiB for an array with shape (500, 300, 300, 3) and data type float32

In [246]:
def train_image_model():
    #(X_train, y_train) = X_train, y_train
    #(X_test, y_test) = X_test, y_test
    X_train = X_train_0.reshape(len(X_train_data_set),300,300,3) 
    X_test = X_test_0.reshape(len(X_test_data_set),300,300,3)
    y_train = to_categorical(y_train_0)
    y_test = to_categorical(y_test_0)
    
    #print(y_train[0])
    
    
    model = Sequential([
                #고정
                Input(shape=(300,300,3), name='input_layer'),
                
                # n_filters * (filter_size + 1) = 32*(9+1) = 320
                Conv2D(32, kernel_size=3, activation='relu', name='conv_layer1'),
                
                
                #Dropout(0.5)
                MaxPooling2D(pool_size=2),
                Conv2D(8, kernel_size=3, activation='relu', name='conv_layer2'),
                MaxPooling2D(pool_size=2),
                Flatten(),
                #Dense(20, activation='softmax', name='output_layer')
        
        
                #고정
                Dense(3, activation='softmax', name='output_layer')
            ])

    model.summary()    
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=15, epochs=3)
    plot_loss_curve(history.history)
    print(history.history)
    print("train loss=", history.history['loss'][-1])
    print("validation loss=", history.history['val_loss'][-1])    
    
    model.save('model-201814132')
    
    return model

if __name__ == '__main__' :
    train_mnist_model()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_layer1 (Conv2D)         (None, 298, 298, 4)       112       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 149, 149, 4)       0         
_________________________________________________________________
conv_layer2 (Conv2D)         (None, 147, 147, 8)       296       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 73, 73, 8)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 42632)             0         
_________________________________________________________________
output_layer (Dense)         (None, 3)                 127899    
Total params: 128,307
Trainable params: 128,307
Non-trainable params: 0
________________________________________________

ResourceExhaustedError:  OOM when allocating tensor with shape[15,298,298,4] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node sequential_7/conv_layer1/Relu (defined at <ipython-input-202-6e80fcfe988d>:36) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_10851]

Function call stack:
train_function


In [236]:
print(X_test.shape)

(10000, 28, 28)


In [ ]:

def train_image_model():
    (X_train, y_train), (X_test, y_test) = ************데이터 어떻게 불러올거야? 
    #X_train = X_train.reshape(60000,28,28,3) 
    #X_test = X_test.reshape(10000,28,28,3)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
#     print(y_train[1563])
    
    
    #옵티마이저, loss , 변경해보기
    model.compile(optimizer= 'adam', loss='categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(X_train,y_train, validation_data=(X_test, y_test), batch_size=20, epochs = 3)
    
    plot_loss_curve(history.history)
    print(history.history)
    print('train loss =', history.history['loss'][-1])
    print('validation loss =', history.history['val_loss'][-1])

    model.save('model-201814132')
    return model


예측


In [230]:
test_id = -1
if test_id <0 :
    from random import randrange
    test_sample_id = randrange(30)
else:
    test_sample_id = test_id

test_image =X_test[test_sample_id]
print(X_test)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [228]:
def predict_image_sample(model,X_test, y_test, test_id= -1 ):
    if test_id <0 :
        from random import randrange
        test_sample_id = randrange(30)
    else:
        test_sample_id = test_id
    
    test_image =X_test[test_sample_id]
    print(X_test)
#     plt.imshow(test_image, cmap = 'gray')
    
    test_image = test_image.reshape(1,8,49,3)
    
    y_actual = y_test[test_sample_id]
    #print('y_actual number = ', y_actual)
    
    y_pred = model.predict(test_image)
    #print("y_pred = ", y_pred)
    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    #print('y_pred number = ', y_pred)
    
    return y_actual , y_pred_num
    
predict_image_sample(model,X_test, y_test)

ValueError: cannot reshape array of size 784 into shape (1,8,49,3)

실행

In [212]:

def predict_image_sample(model,X_test, y_test, test_id= -1 ):
    if test_id <0 :
        from random import randrange
        test_sample_id = randrange(10000)
    else:
        test_sample_id = test_id
    
    test_image =X_test[test_sample_id]
#     plt.imshow(test_image, cmap = 'gray')
    
    test_image = test_image.reshape(1,300,300,3)
    
    y_actual = y_test[test_sample_id]
    print('y_actual number = ', y_actual)
    
    y_pred = model.predict(test_image)
    print("y_pred = ", y_pred)
    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    print('y_pred number = ', y_pred)
    return y_actual , y_pred_num
'''    
if __name__ == '__main__' :
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    model = load_model('mnist.model')
    predict_image_sample(model,X_test, y_test)
'''    
    
if __name__ == '__main__' :
    n = 0
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    model = load_model('mnist.model')
#     predict_image_sample(model,X_test, y_test)
    
    for i in range(700):
        y_actual , y_pred_num = predict_image_sample(model,X_test, y_test,test_id = i)
        if y_actual == y_pred_num:
            n += 1 
        else : pass
print('전체 횟수 :', i,' '*10,'예측 성공 횟수',n,'\n'*2 ,'정확도 = ', (n/i)*100)

500
(850, 300, 300, 3)
(151, 300, 300, 3)
(850,)
(151,)
[[[[184. 146. 109.]
   [185. 147. 110.]
   [185. 147. 110.]
   ...
   [ 73.  83.  75.]
   [ 71.  80.  75.]
   [ 72.  81.  76.]]

  [[174. 137.  95.]
   [175. 138.  96.]
   [177. 140.  98.]
   ...
   [ 74.  85.  77.]
   [ 67.  78.  72.]
   [ 66.  75.  70.]]

  [[158. 121.  77.]
   [159. 122.  78.]
   [160. 123.  78.]
   ...
   [ 64.  81.  71.]
   [ 60.  76.  66.]
   [ 64.  75.  67.]]

  ...

  [[ 10.   6.   3.]
   [ 14.   9.   6.]
   [ 16.   3.   0.]
   ...
   [184. 178. 166.]
   [176. 172. 161.]
   [173. 169. 158.]]

  [[ 11.   9.  10.]
   [ 14.   9.   6.]
   [ 23.  10.   2.]
   ...
   [179. 175. 163.]
   [178. 174. 163.]
   [175. 173. 161.]]

  [[ 23.  23.  25.]
   [ 16.  10.  10.]
   [ 24.  11.   3.]
   ...
   [179. 175. 163.]
   [179. 177. 165.]
   [179. 176. 167.]]]


 [[[ 29.  19.   7.]
   [ 30.  20.   8.]
   [ 28.  18.   6.]
   ...
   [126. 114. 102.]
   [125. 113. 101.]
   [128. 116. 104.]]

  [[ 29.  19.   7.]
   [ 31.  21

ValueError: too many values to unpack (expected 2)